# Building Realistic Example Data
This __optional bonus assignment__ is worth up to 5% of your final grade. It must be handed in by directly mailing the instructor (Christopher Brooks, brooksch@umich.edu) the assignment no later than Sunday September 27th at 11:59pm EST. This is not an all-or-nothing assignment, partial grades will be provided as appropriate.

## Assignment Overview
A constant need I have when teaching pandas is finding compelling example data to work from. Good example data is real-world, messy enough to need some manipulation, and fits reasonable constraints for a given problem. For instance, if I want to demonstrate joining multiple `DataFrame` together I might want one which is about people and one which is about purchases, where every person has an identifier and a bunch of personal information, and every purchase is linked to a given person. This is much more compelling then a bunch of random `np.ndarray` lists that I create inline while trying to give a lecture!

In addition, I'm taken by domain specific languages, and in this part of the assignment you are required to build a processor for a simple domain specific language I have invented for the purpose of describing pandas `DataFrame` structures! It is expected that you will demonstrate your knowledge of regex here in particular.

Here's an example of the language I've created for this part of the assignment:
```
persons
-------
first_name
last_name*
phone_number
random_number(5) as customer_number [1]*

purchases
---------
isbn10
credit_card_full
random_number(3) as price
random_number(5) as customer_number [1]
```

In this example I describe two `DataFrame` objects by underlying a string with two or more hyphens. The string (`persons` and `purchases`) should be used as the variable name for the `DataFrame` objects created, and the language will always separate multiple `DataFrame` definitions by whitespace and hyphens as shown. Each column in the `DataFrame` is described on its own line with a string (e.g. `first_name*`). The string defines the column as follows:

1. The first word (e.g. `first_name` or `random_number(3)`) describes a function  and optional set of parameters to be called against a common `faker` object (an instance of the `Faker` class) for each entry in the resulting `DataFrame`. For instance, a value of `isbn10` implies that `faker.isbn10()` be called (note the default parameters), while a value of `random_number(5)` implies that `faker.random_number(5)` be called (with my supplied parameters).
2. The first word *may* be followed by some whitespace and then an `as` statement. The `as` statement denotes that the following word be used as the name of the column. For instance, `random_number(3) as price` means that I'm looking for a column named `price` where every instance in the column is of a `faker.random_number(3)` invocation.
3. If there is no `as` statement the name of the column should be the name of the function (with no parameters) I supplied, e.g. `first_name`.
4. The definition may include a reference in the format `[#]` where the # sign is any number. This reference will be used across tables to show that the data in those tables should be of a similar set of values. This is so I can join between tables, which is a common need to demonstrate. In the example given I want each of the tables to have a column called `customer_number` where the data values in the column are such that `set(column1)==set(column2)`. Note that this doesn't mean the columns should be the same (they shouldn't), but just that they should include only 100% overlapping data. See point 6 for data distribution. **Clarification: In the example here, all of the `customer_number` values in `persons` should be unique (hence the `*`), but in the referencing collumn `purchases[customer_number]` the uniqueness is relaxed. This means that there does not have to be a 1:1 mapping as implied, and that it is a 1:many mapping, and in this case since `purchases` has repeated data you could verify this with `set(purchases['customer_number']).issubset(set(persons['customer_number']))`.**
5. The sentence may end with a `*`. This indicates that the column described should be made up of unique data (no repeated elements). For instance, you wouldn't want a customer to be accidently assigned the customer number of another person! In the example above I decided I wanted the `last_name` of the persons and their `customer_number` to be unique in that table.
6. By default a column should have 25% repeated data. e.g. something close to `len(column)==len(set(column))*1.25`. This lets me demonstrate operations such as left joins easily.
7. By default, the length of each `DataFrame` created should be 99 items. This is both reasonable for most demonstrations, and a homage to The Great One.
8. The functionality described should be executed in a cell magic function called `%%fakedata`, where the remainder of the cell is the definition in plain text. See https://ipython.readthedocs.io/en/stable/config/custommagics.html for more details.

## An attempt at a more formal grammar

```
function_to_call  ::= <wordcharacters>
parameters        ::= "" | "(" ( wordcharacters | number ) ")"
as_name           ::= "" | "as" <whitespace> <wordcharacters>
column_name       ::= as_name | function_to_call
reference         ::= "" |  "[" number "]"
unique_mark       ::= "" | "*"
column_definition ::= <function_to_call> <parameters> <whitespace> \
                      <as_name> <whitespace> <reference> <unique_mark>
df_sep            ::= "--" ("-"*)
df_definition     ::= <wordcharacters> <newline> <df_sep> <newline> \
                      (<column_definition>*) <newline> <newline>
language_spec     ::= <def_definition>*
```


## Background: What is this `Faker` class?
The `Faker` class defines a number of great functions that generate realistic data. The way it works is that you create a new instance of `Faker` with no parameters, then call various methods on that object which are predefined at https://faker.readthedocs.io/en/stable/

This is demonstrated below showing a single entry into a `DataFrame` using the description above.

In [1]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
import pandas as pd
import numpy as np
from faker import Faker
import re
import random

# Config for generator
# Please tweak these numbers sensibly, keeping in mind unique faker values and reference linking
DEFAULT_DF_SIZE = 99     # default number of rows per DataFrame
ORPHANED_UNIQUES = 0.2   # % of uniques references which won't be used in non-unique references
REPEAT_WEIGHTS = 1000    # Weight of chance for repeated values.  1 would be equal weights
# I haven't really done the detailed math, on the higher end or larger DataFrame size, so maybe bigger variance end up
# wouldn't be so big afterall.
IS_DEBUG = True          # debug flag for a few print statements.  not sure if and how __debug__ should be used instead?

assert (REPEAT_WEIGHTS >= 1) and type(REPEAT_WEIGHTS) == int, "REPEAT_WEIGHTS must be type int and >= 1"

# Global variables
fake = Faker()

# This unique mapping helps maps a faker function to a provider's ordered dict
# This is used for efficiently generating large amount of unique data if the data is taken from a dict
# For example, fake.first_name() can possibly map to faker.providers.person.en_US.Provider.first_names
unique_mapping = {
    # There are probably more, but I'll stick to first_name() and last_name() for now
    'first_name':'first_names',
    'last_name':'last_names'
}

# Helper functions for casting parameters to int, float or string
def cast_parameter(x):
    if x is None:
        return x
    elif type(x) != str:
        raise ValueError('Input must be a string or None')
    try:
        a = float(x)
        b = int(a)
    except ValueError:
        pass
    else:
        if a == b:
            return b
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        return a
    
def dprint(s):
    if IS_DEBUG:
        print(s)

In [2]:
# This function generate a singular value based on the Faker function and reference used
# Uniqueness is checked against the set "set_for_uniqueness", a local reference for the caller is needed
# Pass None to set_for_uniqueness for no uniqueness requirement

def gen_data(_, function_name,parameter,set_for_uniqueness):
    # QUESTION: not sure if passing by set() by reference for set_for_uniqueness will mess up by panda's
    #           concurrency. It SEEMS OK, but have not tested rigorously
    # TODO:     parameter currently only takes 1 variable, need to convert to support multiple?
    # TODO:     For some function_name like first_name, we can directly get the Provider list of possible
    #           values and get a subset from it rather than repeatedly generating them
    #               from faker.providers.person.en import Provider
    #               set(Provider.first_names)
    func = getattr(fake, function_name)
    parameter = cast_parameter(parameter)
    while(True):
        value = func() if parameter is None else func(parameter)
        if type(set_for_uniqueness) is set:
            if value not in set_for_uniqueness:
                set_for_uniqueness.add(value)
                return value
        else:
            return value

In [3]:
# This function generate a list of unique values from for supported functions listed in unique_mapping
# The data is taken directly from the faker providers.

def gen_unique_data(function_name, count):

    assert function_name in unique_mapping.keys(), "Unsupported function for gen_unique_data(): {}".format(function_name)
    
    for p in fake.get_providers():
        if function_name in dir(p) and unique_mapping[function_name] in dir(p):
            assert len(getattr(p,"last_names").keys()) >= count, "Insufficient data ({} available) from provider to generate {} \"{}\".".format(len(getattr(p,"last_names").keys()), count, function_name)
            return random.sample(getattr(p,"last_names").keys(), count)
    # explicitly putting this here to remind myself it maybe possible that providers don't have the available data
    # caller should ensure that data is provided and not None
    return None

In [4]:
# This function generates data specifically for reference columns since the data needs to be shared amongst DataFrames

def get_reference_column_data(reference_key, function_name,parameter,is_unique, count, reference_dict):
    if reference_key not in reference_dict.keys():
        new_data = set()

        if function_name in unique_mapping.keys():
            unique_data = gen_unique_data(function_name, count)
            if unique_data is not None:
                new_data = unique_data
        
        while(len(new_data)<count):
            new_data.add(gen_data(None, function_name,parameter,new_data))
        reference_dict[reference_key] = new_data

    if is_unique:
        # if we are just looking for the unique data, we can returned the shuffled set
        return random.sample(reference_dict[reference_key], count)
    else:
        # sample items to exclude some reference values

        unique_count = int(len(reference_dict[reference_key])*(1-ORPHANED_UNIQUES))
        picked_uniques = random.sample(reference_dict[reference_key], min(unique_count, count))
        
        # apart from initial copy how many additional copies needed?
        w = [random.uniform(1,REPEAT_WEIGHTS) for _ in range(unique_count)]
        random_picked = np.random.choice(picked_uniques, count - len(picked_uniques), p = [float(i)/sum(w) for i in w])
        d = picked_uniques + list(random_picked)
        return d

In [5]:
@register_cell_magic
def fakedata(line,cell):
    reference_dict = {}
    
    # Step 1, we parse the input into 2 separate dataframes
    # dataframe_size contains the size for each dataframe
    # dataframe_col_def contains column definitions of each dataframe
    dataframe_size = pd.DataFrame(columns=['df_name', 'size'])
    dataframe_size.set_index('df_name', inplace=True)
    dataframe_col_def = pd.DataFrame(columns=['df_name', 'col_name', 'function', 'parameter', 'as_name', 'reference', 'unique_mark'])
    dataframe_col_def.set_index(['df_name','col_name'], inplace=True)

    # we split the dataframes out to process them one at a time
    regex = re.compile(r'(?P<df_name>[\w]+)(?: \[(?P<df_size>\d+)\])?(?:\n-+\n)(?P<df_def>(?:.+\n?)+)')
    for dfdict in [m.groupdict() for m in regex.finditer(cell.strip())]:
        df_name = dfdict['df_name']
        df_size = int(dfdict['df_size']) if dfdict['df_size'] is not None else DEFAULT_DF_SIZE
        definitions = dfdict['df_def'].strip().split('\n')

        # we can check that there are no duplicated dataframe names
        assert df_name not in dataframe_size.index, "DataFrames with duplicated names found: {}".format(df_name)
        dataframe_size.loc[df_name] = [df_size]
        
        # break down each column of the DataFrame based on provided definitions
        for d in definitions:
            dpattern = r"^(?P<function>\w+)(?:(?:\((?P<parameter>[-+]?\d*\.?\d+|\w+)\))?)(?: as (?P<as_name>\w*))?(?: \[(?P<reference>\d+)\])?(?P<unique_mark>\*)?$"
            dmatch = re.search(dpattern,d).groupdict()
            col_name = dmatch['as_name'] if dmatch['as_name'] is not None else dmatch['function']
            dmatch['unique_mark'] = dmatch['unique_mark'] is not None
        
            # we can check that there are no duplicated dataframe names
            assert (df_name, col_name) not in dataframe_col_def.index, "Columns with duplicated names found in DataFrame \"{}\": {}".format(df_name, col_name)
            dataframe_col_def.loc[(df_name, col_name),dataframe_col_def.columns] = [dmatch['function'], dmatch['parameter'], dmatch['as_name'], dmatch['reference'], dmatch['unique_mark']]
    
    # we don't need as_name anymore since we now have "col_name", so let's drop it
    dataframe_col_def.drop(columns=['as_name'], inplace=True)

    # These are the parsed input
    #dprint(dataframe_size)
    #dprint(dataframe_col_def)
    
    # Step 2, we create the dataframes with no data
    for d in dataframe_size.itertuples():
        df_name = d.Index
        df_size = d.size
        dfr = dataframe_col_def.loc[df_name].reset_index()
        # create DataFrame one by one
        df = pd.DataFrame(index=range(0,df_size), columns=dfr['col_name'].tolist())
        # assign the Dataframe to the global space
        globals()[df_name] = df

    # Step 3, pre-generate all references first
    # for each reference that has  a unique_mark, we generate the exact amount needed to fill its dataframe
    # if no unique, just find the highest
    ref_df = dataframe_col_def.join(dataframe_size, how='inner', on='df_name')
    ref_df = ref_df.groupby(['reference','function','parameter','unique_mark'], as_index=False)[['size']].max()
    ref_check = ref_df[ref_df.reset_index().duplicated(subset=['reference','unique_mark'])]
    ref_df.sort_values(by=['unique_mark'], ascending=False, inplace=True)
    ref_df.drop_duplicates(subset=['reference','function','parameter'], inplace=True)
    #dprint(ref_df)
    assert len(ref_check) == 0, 'Inconsistent "function" and "parameter" definitions for references: {}'.format(ref_check.reference.tolist())
    for r in ref_df.itertuples():
        # Assuming we're generating uniques here so random generation isn't needed.
        get_reference_column_data(r.reference,r.function,r.parameter,r.unique_mark, r.size, reference_dict)
    
    # Step 4, populate all the data
    for c in dataframe_col_def.join(dataframe_size, how='inner', on='df_name').itertuples():
        df_name, col_name = c.Index
        df_size = c.size
        if c.reference is not None:
            # this is a reference column, we need to reference our references of the reference
            # since we're demonstrating One to Many
            globals()[df_name][col_name] = get_reference_column_data(c.reference,c.function,c.parameter,c.unique_mark, df_size, reference_dict)
        else:
            if c.function in unique_mapping.keys():
                unique_data = gen_unique_data(c.function, df_size)
                if unique_data is not None:
                    globals()[df_name][col_name] = unique_data
                    continue
            globals()[df_name][col_name] = globals()[df_name][col_name].apply(gen_data, args=(c.function,c.parameter,set() if c.unique_mark else None))
        
